In [575]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

using DataFrames, DataFramesMeta
using CSV
using Statistics
using StatsBase
using MultivariateStats
using Main

In [576]:
KEY = "2019wk1"
YEAR = "2019"

"2019"

In [577]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 3534×54 DataFrame. Omitted printing of 50 columns
│ Row  │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│      │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├──────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1    │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 2    │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 3    │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 4    │ 0            │ 15         │ Panel         │ Panel         │
│ 5    │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 6    │ 0            │ 3          │ PanelAndCargo │ PanelAndCargo │
│ 7    │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 8

In [578]:
if YEAR == "2019"
    DeepSpace.calculated_columns!(scores)
end
scores[:ownPoints] = scores.totalPoints .- scores.foulPoints .- (3 .* scores.foulCount) .- (9 * scores.techFoulCount)

3534-element Array{Int64,1}:
  73
  51
  80
  57
  80
  26
  75
  44
  64
  43
  55
  53
  58
   ⋮
  45
  54
  48
  32
  60
  39
  67
  44
  39
  44
  50
 -18

In [579]:
(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train, shortevent=true)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Union{Missing, String}["frc1080", "frc1086", "frc1123", "frc116", "frc122", "frc1262", "frc1413", "frc1418", "frc1522", "frc1598"  …  "frc6504", "frc6543", "frc6584", "frc6802", "frc6863", "frc6882", "frc7330", "frc7429", "frc888", "frc977"], Dict("frc888"=>74,"frc1086"=>2,"frc2421"=>24,"frc3359"=>34,"frc3941"=>41,"frc5546"=>51,"frc4242"=>44,"frc1418"=>8,"frc3136"=>30,"frc6802"=>69…))

In [580]:
scores_train = @linq scores |> where(:level .== "qm") |> sort(:event)
teams_train = @linq teams |> where(:level .== "qm") |> sort(:event)

scores_test = @linq scores |> where(:level .!= "qm") |> sort(:event)
teams_test = @linq teams |> where(:level .!= "qm") |> sort(:event)

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train, shortevent=true)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Union{Missing, String}["frc1023", "frc103", "frc1051", "frc108", "frc1080", "frc1086", "frc1102", "frc1123", "frc114", "frc1143"  …  "frc841", "frc86", "frc88", "frc888", "frc900", "frc94", "frc949", "frc95", "frc973", "frc977"], Dict("frc7458"=>665,"frc7298"=>653,"frc293"=>152,"frc5527"=>424,"frc6919"=>599,"frc7033"=>615,"frc3547"=>217,"frc5401"=>408,"frc1942"=>81,"frc7524"=>676…))

In [581]:
cols = Lib.numeric_col_names(scores_train)
y = scores_train[cols] |> df -> convert(Array{Float64,2}, df)
linearComponentsArr = llsq(ß,y; bias=false)
ridgeComponentsArr = ridge(ß,y,1; bias=false)

752×21 Array{Float64,2}:
 0.0  3.09851   7.08322   -0.0290397    …  -0.120028    5.16899     24.1652 
 0.0  3.12333   7.44759    0.138573         0.529812    2.19659     22.3751 
 0.0  3.14808   6.01432    0.0085236       -0.0215812   2.64095     19.9841 
 0.0  3.47705   4.30426   -0.00961452       0.911397    5.04411     13.8517 
 0.0  2.97172   3.97656   -0.0125419        0.559505    2.3366      11.4686 
 0.0  3.35114   4.37163   -0.00623019   …  -0.219893   -0.883082    12.1164 
 0.0  5.19883   7.73174    0.0714837        1.88306     9.54822     23.3022 
 0.0  3.21513   1.3451     0.00426104      -0.0572035   0.523081     5.65082
 0.0  2.34996   8.62203   -0.0442876       -1.00274     3.45062     14.9759 
 0.0  2.58396   3.66847   -0.0252204       -0.290644   -0.412658     8.5123 
 0.0  3.59683   7.55249   -0.0370781    …   0.356335    2.79032     16.4364 
 0.0  3.1266    5.98722   -0.0127407        0.078529    0.500573    12.2588 
 0.0  3.00343   6.79799   -0.0273639        0.42468

In [582]:
event_lengths_tbl = aggregate(scores_train, :event, length)
event_lengths = Dict(event_lengths_tbl.event[i] => event_lengths_tbl.totalPoints_length[i] / 2 for i = 1:size(event_lengths_tbl,1))
timevar = [let
        key = split(scores_train.key[i], "_")[2]
        num_str = split(key, "qm")[end]
        num = parse(Float64, num_str)
        len = event_lengths[scores_train.event[i]]
        (len - num) / len
        end for i = 1:size(ß,1)]
[scores_train.key timevar]

2864×2 Array{Any,2}:
 "2019cadm_qm1_red"     0.985714 
 "2019cadm_qm1_blue"    0.985714 
 "2019cadm_qm10_red"    0.857143 
 "2019cadm_qm10_blue"   0.857143 
 "2019cadm_qm11_red"    0.842857 
 "2019cadm_qm11_blue"   0.842857 
 "2019cadm_qm12_red"    0.828571 
 "2019cadm_qm12_blue"   0.828571 
 "2019cadm_qm13_red"    0.814286 
 "2019cadm_qm13_blue"   0.814286 
 "2019cadm_qm14_red"    0.8      
 "2019cadm_qm14_blue"   0.8      
 "2019cadm_qm15_red"    0.785714 
 ⋮                               
 "2019wamou_qm66_red"   0.0294118
 "2019wamou_qm66_blue"  0.0294118
 "2019wamou_qm67_red"   0.0147059
 "2019wamou_qm67_blue"  0.0147059
 "2019wamou_qm68_red"   0.0      
 "2019wamou_qm68_blue"  0.0      
 "2019wamou_qm7_red"    0.897059 
 "2019wamou_qm7_blue"   0.897059 
 "2019wamou_qm8_red"    0.882353 
 "2019wamou_qm8_blue"   0.882353 
 "2019wamou_qm9_red"    0.867647 
 "2019wamou_qm9_blue"   0.867647 

In [583]:
ßspecial = [ß timevar]
specialComponents = llsq(ßspecial,y; bias=false)

753×21 Array{Float64,2}:
 0.0   3.74449   8.53458  -0.0321876   …  -0.204082     5.62427    28.0949 
 0.0   3.76609   8.9895    0.15224         0.53301      2.43728    26.0689 
 0.0   3.75819   7.1438    0.0137316      -0.198531     2.86532    23.6255 
 0.0   4.43164   6.07839  -0.0110993       1.10179      5.96471    17.6607 
 0.0   3.61294   5.25792  -0.0133102       0.56933      2.64877    14.3573 
 0.0   4.06478   5.85715  -0.0045856   …  -0.252378    -0.730766   15.3588 
 0.0   6.15546   9.37998   0.081163        2.054       10.6508     27.4179 
 0.0   3.93271   2.48792   0.00770249     -0.0626202    1.01305     8.35283
 0.0   2.82498  10.1231   -0.0494314      -1.17101      3.74173    17.7178 
 0.0   3.17351   4.80635  -0.0275532      -0.353089    -0.391092   10.7768 
 0.0   4.29029   9.13871  -0.0406148   …   0.337362     3.18981    19.768  
 0.0   3.84586   7.60615  -0.013174        0.0326811    0.730846   15.4295 
 0.0   3.58327   8.25823  -0.0308372       0.388668     6.60164

In [584]:
tSpecial = [teamnums ; "special"]
linearOPR = DataFrame([teamnums linearComponentsArr], [:team ; cols])
ridgeOPR  = DataFrame([teamnums ridgeComponentsArr], [:team ; cols])
specialOPR = DataFrame([tSpecial specialComponents], [:team ; cols])

,team,adjustPoints,autoPoints,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,hatchPanelPoints,rp,sandStormBonusPoints,techFoulCount,teleopPoints,totalPoints,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints,ownPoints
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,frc1023,0.0,3.74449,8.53458,-0.0321876,-0.0163787,-0.0113106,-0.0306074,0.899132,10.388,0.679145,4.9778,1.57517,3.74449,-0.0398019,23.9004,28.544,5.62141,0.206933,-0.204082,5.62427,28.0949
2,frc103,0.0,3.76609,8.9895,0.15224,0.000612231,0.000462723,-0.048465,2.89509,9.86087,0.694195,2.83466,1.71419,3.76609,-0.0524897,21.685,28.3462,0.883313,1.02095,0.53301,2.43728,26.0689
3,frc1051,0.0,3.75819,7.1438,0.0137316,0.0125371,0.000530971,0.120512,1.18153,9.25707,0.620487,2.99778,1.62903,3.75819,-0.0922497,19.3987,24.3384,0.0791198,2.98473,-0.198531,2.86532,23.6255
4,frc108,0.0,4.43164,6.07839,-0.0110993,-0.0120868,0.000438967,0.825078,1.50109,5.65394,0.313737,4.48723,1.1027,4.43164,0.0572512,16.2196,22.1523,4.60053,0.262397,1.10179,5.96471,17.6607
5,frc1080,0.0,3.61294,5.25792,-0.0133102,-0.0153224,0.000465413,-0.1408,-0.365744,3.9205,0.0742411,0.977378,0.227504,3.61294,-0.0184606,10.1558,13.403,1.3051,0.774337,0.56933,2.64877,14.3573
6,frc1086,0.0,4.06478,5.85715,-0.0045856,0.00762966,0.000507049,-0.070772,0.176941,3.9397,0.00883799,1.0536,-0.0283372,4.06478,-0.0256954,10.8505,15.0922,-0.0607307,-0.417658,-0.252378,-0.730766,15.3588
7,frc1102,0.0,6.15546,9.37998,0.081163,0.000652511,0.0799191,0.221827,1.01019,6.74225,0.304021,5.62066,1.05552,6.15546,-0.0205543,21.7429,28.9085,5.03703,3.55977,2.054,10.6508,27.4179
8,frc1123,0.0,3.93271,2.48792,0.00770249,0.000633163,0.000478543,0.517462,0.115823,2.64579,0.0919513,0.274709,0.103337,3.93271,-0.0626756,5.40842,9.45695,1.06873,0.00694166,-0.0626202,1.01305,8.35283
9,frc114,0.0,2.82498,10.1231,-0.0494314,-0.0137637,-0.0109913,-0.0307875,0.418652,2.74206,0.050828,1.69389,0.396181,2.82498,-0.0268154,14.5591,17.8027,4.05412,0.858621,-1.17101,3.74173,17.7178


In [585]:
[teamnums linearOPR.totalPoints ridgeOPR.totalPoints specialOPR.totalPoints[1:end-1]] |> DataFrame |> s -> sort(s, :x4, rev=true)

,x1,x2,x3,x4
,Any,Any,Any,Any
1,frc330,43.1368,39.5579,45.6332
2,frc346,38.6418,36.0532,41.3605
3,frc973,37.334,34.8538,39.3856
4,frc2910,35.8016,33.5799,38.4041
5,frc3538,34.0104,31.9334,36.7913
6,frc118,34.0375,31.7742,36.5683
7,frc1690,32.8817,30.8064,35.6639
8,frc5883,32.9069,32.2708,35.5483
9,frc614,32.5854,30.6659,35.2877


In [586]:
(ßtest, _, _) = Lib.opr_ß(scores_test, teams_test ; teamnums = teamnums)
scaledToTotalMean(df, col) = (col ./ mean(col)) .* mean(df.totalPoints)

specialTotal = specialOPR.totalPoints[1:end-1]
specialOwn = specialOPR.ownPoints[1:end-1]

yhat4 = ßtest * linearOPR.totalPoints
yhat1 = ßtest * scaledToTotalMean(linearOPR, ridgeOPR.ownPoints)
yhat2 = ßtest * scaledToTotalMean(linearOPR, ridgeOPR.totalPoints)
yhat3 = ßtest * scaledToTotalMean(linearOPR, linearOPR.ownPoints)
yhat5 = ßtest * specialTotal
yhat6 = ßtest * specialOwn #(specialOwn ./ mean(specialOwn) .* mean(specialTotal))
results = DataFrame(key=scores_test.key, y=scores_test.totalPoints, ridgeOwn=yhat1, ridgeTotal=yhat2, linOwn=yhat3, linTotal=yhat4, specialTotal=yhat5, specialOwn=yhat6)

,key,y,ridgeOwn,ridgeTotal,linOwn,linTotal,specialTotal,specialOwn
,String⍰,Int64⍰,Float64,Float64,Float64,Any,Any,Any
1,2019cadm_f1m1_red,66,65.0464,66.0788,65.9546,67.1981,74.9472,66.2753
2,2019cadm_f1m1_blue,70,61.5841,60.9883,61.8344,61.4234,69.5722,63.0126
3,2019cadm_f1m2_red,75,65.0464,66.0788,65.9546,67.1981,74.9472,66.2753
4,2019cadm_f1m2_blue,56,61.5841,60.9883,61.8344,61.4234,69.5722,63.0126
5,2019cadm_f1m3_red,71,65.0464,66.0788,65.9546,67.1981,74.9472,66.2753
6,2019cadm_f1m3_blue,43,61.5841,60.9883,61.8344,61.4234,69.5722,63.0126
7,2019cadm_qf1m1_red,73,65.0464,66.0788,65.9546,67.1981,74.9472,66.2753
8,2019cadm_qf1m1_blue,47,50.4879,44.6817,50.9571,44.7384,52.8735,53.3377
9,2019cadm_qf1m2_red,69,65.0464,66.0788,65.9546,67.1981,74.9472,66.2753


In [587]:
cols2 = [:ridgeOwn, :ridgeTotal, :linOwn, :linTotal, :specialTotal, :specialOwn]
res2 = cols2 .|> c -> √mean((results.y .- results[c]).^2)
res2 = DataFrame(types=cols2, rmse=res2)

,types,rmse
,Symbol,Float64
1,ridgeOwn,15.0601
2,ridgeTotal,14.8913
3,linOwn,15.1034
4,linTotal,14.753
5,specialTotal,13.8963
6,specialOwn,14.1843


In [592]:
out = DataFrame(team=teamnums, opr=linearOPR.ownPoints, specialopr=specialOwn)
sort(out, :specialopr, rev=true)

,team,opr,specialopr
,String⍰,Any,Any
1,frc330,40.0916,42.5703
2,frc346,36.3876,39.087
3,frc2910,34.6477,37.2317
4,frc973,35.1614,37.1984
5,frc2200,31.6619,34.346
6,frc1690,31.1361,33.8985
7,frc3538,30.7126,33.4737
8,frc5883,30.8075,33.4301
9,frc1519,30.845,33.2818


In [607]:
using CSV
CSV.write("../output/$(KEY)_components_linear.csv", linearOPR)
CSV.write("../output/$(KEY)_components_special.csv", specialOPR)

"../output/2019wk1_components_special.csv"